In [1]:
from datasets import load_dataset
import pandas as pd

# Load the dataset
ds = load_dataset("honicky/short_childrens_stories_with_labeled_character_names")

# Convert the training set to a DataFrame
df_train = pd.DataFrame(ds['train'])
df_train.head()

C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,story,characters
0,"In a small town named Mathville, lived two be...","Algy,Sammy"
1,"One sunny day, Alex and Jamie decided to go f...","Alex,Jamie"
2,"In the hot, dusty town of Mississippi during ...","Bowie,Chicamaw,T-Dub,Mattie,Lula,Keechie"
3,"In the bustling city of Philadelphia, Pennsyl...","Timmy,Sarah,Jimmy Lyggett SR"
4,"It was a sunny day in Home Runville, a town f...","Tommy,Alexa,Mark"


In [17]:
df_train[["story","characters"]].loc[1550]

story          In a small town named Chillyville, way up nor...
characters                        Almond,Aloi,Wise Old Squirrel
Name: 1550, dtype: object

In [3]:
# Average number of words
avg_word_len = sum(len(story.split()) for story in df_train["story"]) / len(df_train["story"])
print("Average story length (words):", avg_word_len)

Average story length (words): 300.8056414219474


In [5]:
avg_char_len = sum(len(story) for story in df_train["story"]) / len(df_train["story"])
print("Average story length (Characters):", avg_char_len)

Average story length (Characters): 1868.253091190108


In [9]:
import re

# Use regex to split on sentence-ending punctuation followed by space and uppercase letter
def count_sentences(text):
    return len(re.findall(r'[^.!?]+[.!?]', text))

avg_sent_len = sum(count_sentences(story) for story in df_train["story"]) / len(df_train["story"])
print("Average story length (sentences):", avg_sent_len)


Average story length (sentences): 21.48724884080371


In [2]:
# Save to CSV
df_train.to_csv("short_childrens_stories_train.csv", index=False)

In [3]:
df_train.shape

(2588, 2)

In [ ]:
def build_prompt(story_text):
    return f"""
Your task is to reformat the given story into a dialogue/script format, without making **any changes** to the original story.

---

🧾 **Reformatting Instructions ONLY**:
- Do **not** rewrite, add, remove, or alter any parts of the original story.
- You are **not allowed to enhance** or improve the story — preserve the wording **exactly as it is**.
- The only allowed change is formatting the content using the structure below.

---

🔑 **Also extract 5 keywords**:
- Provide **EXACTLY 5 keywords** (no more, no less), capturing the core of the story.
- Must be a **comma-separated list** of **nouns or key concepts**.
- Begin the line with: **Keywords:**
- ❗ If there are more or less than 5 keywords, the response is rejected.

---

🎭 **Script Format (Only reformatting)**:
- Format each sentence as: `[SpeakerName](emotion): dialogue`
- Do not add any new dialogue or change punctuation.
- Use "Narrator" as the speaker for descriptive lines.
- Use **EXACTLY ONE** emotion from the list below per line.

✅ Allowed Emotions:
happy, sad, angry, fearful, calm, surprised, neutral

❌ Disallowed Emotions:
Any other emotion (e.g., "confused", "excited", "bored", etc.) will invalidate the response.

---

🧪 **Example Input**:
Once upon a time there was a boy named Rob. He was very smart. He sat by a river and thought: "What if I could fly?"

🧾 **Example Output**:
**Keywords:** Rob, river, fly, thought, boy  
[Narrator](neutral): Once upon a time there was a boy named Rob.  
[Narrator](calm): He was very smart.  
[Narrator](calm): He sat by a river and thought:  
[Rob](surprised): What if I could fly?

---

⚠️ **ABSOLUTE RULES**:
- Do not summarize, paraphrase, or creatively rewrite any part.
- No extra words or story logic — output must **match the original story exactly**, just reformatted.

Now reformat the story below without changing anything:

{story_text}
"""

In [ ]:
import subprocess
import json
import pandas as pd

def run_with_ollama(prompt):
    # Use subprocess to talk to Ollama
    result = subprocess.run(
        ['ollama', 'run', 'llama2'],
        input=prompt.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    return result.stdout.decode()

# Load your DataFrame
df = pd.read_csv("short_childrens_stories_train.csv")

output_file = "processed_stories_local_llama3.jsonl"
done_indices = set()

try:
    with open(output_file, "r") as f:
        for line in f:
            obj = json.loads(line)
            done_indices.add(obj["index"])
except FileNotFoundError:
    pass

for idx, row in df.iterrows():
    if idx in done_indices:
        continue

    prompt = build_prompt(row["story"])
    try:
        output = run_with_ollama(prompt)

        lines = output.strip().splitlines()
        keywords = []
        script_lines = []

        for i, line in enumerate(lines):
            if "keywords:" in line.lower():
                keyword_line = line.split(":", 1)[1]
                raw_keywords = keyword_line.split(",")
                keywords = [kw.strip(" *-_.").strip() for kw in raw_keywords if kw.strip()]
                script_lines = lines[:i] + lines[i+1:]
                break
        else:
            script_lines = lines

        formatted_script = "\n".join(script_lines).strip()

        with open(output_file, "a") as f:
            json.dump({
                "index": idx,
                "story": row["story"],
                "keywords": keywords,
                "script": formatted_script
            }, f)
            f.write("\n")

        print(f"✅ Saved story {idx}")
    except Exception as e:
        print(f"❌ Failed at index {idx}: {e}")


✅ Saved story 0
✅ Saved story 1
✅ Saved story 2
✅ Saved story 3
✅ Saved story 4
✅ Saved story 5
✅ Saved story 6
✅ Saved story 7
✅ Saved story 8
✅ Saved story 9
✅ Saved story 10
✅ Saved story 11
✅ Saved story 12
✅ Saved story 13
✅ Saved story 14
✅ Saved story 15
✅ Saved story 16
✅ Saved story 17
✅ Saved story 18
✅ Saved story 19
✅ Saved story 20
✅ Saved story 21
✅ Saved story 22
✅ Saved story 23
✅ Saved story 24
✅ Saved story 25
✅ Saved story 26
✅ Saved story 27
✅ Saved story 28
✅ Saved story 29
✅ Saved story 30
✅ Saved story 31
✅ Saved story 32
✅ Saved story 33
✅ Saved story 34
✅ Saved story 35
✅ Saved story 36
✅ Saved story 37
✅ Saved story 38
✅ Saved story 39
✅ Saved story 40
✅ Saved story 41
✅ Saved story 42
✅ Saved story 43
✅ Saved story 44
✅ Saved story 45
✅ Saved story 46
✅ Saved story 47
✅ Saved story 48
✅ Saved story 49
✅ Saved story 50
✅ Saved story 51
✅ Saved story 52
✅ Saved story 53
✅ Saved story 54
✅ Saved story 55
✅ Saved story 56
✅ Saved story 57
✅ Saved story 58
✅ Saved

finetune_data.jsonl for storyGeneration model

In [1]:
import json

input_path = "processed_stories_local.jsonl"
output_path = "finetune_data.jsonl"

with open(input_path, "r") as infile, open(output_path, "w") as outfile:
    for line in infile:
        data = json.loads(line)

        keywords = ", ".join(data.get("keywords", []))
        script = data.get("script", "").strip()

        if keywords and script:
            formatted = {
                "prompt": f"Keywords: {keywords}\n\nWrite a dialogue-based story script:\n",
                "completion": script
            }
            json.dump(formatted, outfile)
            outfile.write("\n")

Evaluation

In [5]:
import re
import json

input_path = "processed_stories_local_llama2.jsonl"

allowed_emotions = {"happy", "sad", "angry", "fearful", "calm", "surprised", "neutral"}
script_line_pattern = re.compile(r"^\[(.*?)\]\((.*?)\):\s*(.+)$")

total_stories = 0
valid_format_count = 0
invalid_keyword_count = 0
invalid_emotion_count = 0

with open(input_path, "r") as infile:
    for line in infile:
        total_stories += 1
        data = json.loads(line)
        keywords = data.get("keywords", [])
        script = data.get("script", "").strip()

        # Check keyword count
        if len(keywords) != 5:
            invalid_keyword_count += 1

        # Check script format and emotion validity
        valid_lines = 0
        for line in script.splitlines():
            match = script_line_pattern.match(line)
            if match:
                speaker, emotion, _ = match.groups()
                if emotion in allowed_emotions:
                    valid_lines += 1
                else:
                    invalid_emotion_count += 1
                    break
            else:
                invalid_emotion_count += 1
                break

        if len(keywords) == 5 and valid_lines == len(script.splitlines()):
            valid_format_count += 1

# Print summary
print({
    "Total Processed Stories": total_stories,
    "Valid Script Format Stories": valid_format_count,
    "Stories with Invalid Keyword Count": invalid_keyword_count,
    "Stories with Invalid Emotion Tags or Format": invalid_emotion_count
})

{'Total Processed Stories': 1079, 'Valid Script Format Stories': 7, 'Stories with Invalid Keyword Count': 1020, 'Stories with Invalid Emotion Tags or Format': 1060}
